In [1]:
import numpy as np
import torch
from torch.nn.functional import conv2d as libConv2d
from torch.nn.functional import conv_transpose2d as libConv2dT
import warnings

In [9]:
def custom_conv_transpose(matrix, 
                          in_channels, 
                          out_channels, 
                          kernel_size,
                          stride=1, 
                          padding=0, 
                          output_padding=0, 
                          dilation=1,
                          bias=True, 
                          padding_mode='zeros'):
    if bias:
        bias_val = torch.rand(out_channels) 
    else:
        bias_val = torch.zeros(out_channels)

    if type(kernel_size) == int:
        weights = torch.rand(in_channels, out_channels, kernel_size, kernel_size)  
    else:
        weights = torch.rand(in_channels, out_channels, *kernel_size)
    res_tensor = []
    for l in range(out_channels):
        feature_map = torch.zeros((matrix.shape[1] - 1) * stride + dilation * (kernel_size - 1) + 1,
                                  (matrix.shape[2] - 1) * stride + dilation * (kernel_size - 1) + 1)

        for c in range(in_channels):
            for i in range(0, matrix.shape[1]):
                for j in range(0, matrix.shape[2]):
                    val = matrix[c][i][j]
                    proizv = val * weights[c][l]
                    zero_tensor = torch.zeros((weights.shape[2] - 1) * dilation + 1,
                                              (weights.shape[3] - 1) * dilation + 1)
                    for a in range(0, zero_tensor.shape[0], dilation):
                        for b in range(0, zero_tensor.shape[1], dilation):
                            zero_tensor[a][b] = proizv[a // dilation][b // dilation]
                    res = np.add(zero_tensor, feature_map[i * stride:i * stride + (weights.shape[2] - 1) * dilation + 1,
                                 j * stride:j * stride + (weights.shape[3] - 1) * dilation + 1])
                    feature_map[i * stride:i * stride + (weights.shape[2] - 1) * dilation + 1,
                    j * stride:j * stride + (weights.shape[3] - 1) * dilation + 1] = res

        res_tensor.append(np.add(feature_map, np.full(feature_map.shape, bias_val[l])))

    for t in range(len(res_tensor)):
        if output_padding > 0:
            pad_func = torch.nn.ConstantPad1d((0, output_padding, 0, output_padding), 0)
            res_tensor[t] = pad_func(res_tensor[t])

        res_tensor[t] = res_tensor[t][0 + padding:res_tensor[t].shape[0] - padding,
                                      0 + padding:res_tensor[t].shape[1] - padding]

    return res_tensor, weights, torch.tensor(bias_val)

def compare_results(tensor, in_channels, out_channels, kernel_size, stride, padding, output_padding, dilation, bias=True, padding_mode='zeros'):
    my_res, kernel, bias_val = custom_conv_transpose(
        tensor,
        in_channels=in_channels, out_channels=out_channels,
        kernel_size=kernel_size, stride=stride,
        padding=padding, output_padding=output_padding,
        dilation=dilation, bias=bias,
        padding_mode=padding_mode,
    )

    torch_function = torch.nn.ConvTranspose2d(
        in_channels=in_channels, out_channels=out_channels,
        kernel_size=kernel_size, stride=stride,
        padding=padding, output_padding=output_padding,
        dilation=dilation, bias=bias,
        padding_mode=padding_mode,
    )
    torch_function.weight.data = kernel
    torch_function.bias.data = bias_val

    result = str(np.round([tensor.tolist() for tensor in my_res], 2))
    torch_res = str(np.round(torch_function(tensor).data.numpy(), 2))
    return result == torch_res

warnings.filterwarnings('ignore')

### Проверяем работоспособность функции:

In [10]:
tensor1 = torch.rand(8, 5, 6)
print(compare_results(tensor1, in_channels=8, out_channels=2, kernel_size=3, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros'))

tensor2 = torch.rand(3, 28, 28)
print(compare_results(tensor2, in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0, output_padding=0, dilation=3, bias=True, padding_mode='zeros'))

tensor3 = torch.rand(5, 6, 6)
print(compare_results(tensor3, in_channels=5, out_channels=1, kernel_size=3, stride=3, padding=5, output_padding=2, dilation=1, bias=True, padding_mode='zeros'))

tensor4 = torch.rand(1, 1, 1)
print(compare_results(tensor4, in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros'))

True
True
True
True


### Дополнительное задание:
#### Реализовать алгоритм работы транспонированной свертки, через алгоритм двумерной свертки, реализованный в первой лабораторной. Необходимо перерассчитать входные параметры.


In [11]:
def myfunc_Conv2D(input, 
                  in_channels, 
                  out_channels, 
                  kernel_size,
                  transp_stride=1, 
                  padding=0, 
                  dilation=1, 
                  bias=True,
                  padding_mode='zeros'):
  stride = 1

  pad = kernel_size - 1
  result_input = []
  for matr in input:
    zero_tensor = np.zeros((((matr.shape[0] - 1) * (transp_stride) + 1), ((matr.shape[1] - 1) * (transp_stride) + 1)))
    for a in range(0, zero_tensor.shape[0], transp_stride):
      for b in range(0, zero_tensor.shape[1], transp_stride):
        zero_tensor[a][b] = matr[a // (transp_stride)][b // (transp_stride)]

    pad_matr = np.pad(zero_tensor, pad_width=pad, mode='constant')
    result_input.append(pad_matr)
  input = torch.tensor(result_input)

  if bias:
    bias_val = torch.rand(out_channels)
  else:
    bias_val = torch.zeros(out_channels)
    
  match padding_mode:
    case 'zeros':
      pad = torch.nn.ZeroPad2d(padding)
      input = pad(input)
    case 'reflect':
      pad = torch.nn.ReflectionPad2d(padding)
      input = pad(input)
    case 'replicate':
      pad = torch.nn.ReplicationPad2d(padding)
      input = pad(input)
    case 'circular':
      pad = torch.nn.CircularPad2d(padding)
      input = pad(input)

  weights = np.array(torch.rand(out_channels, in_channels, kernel_size, kernel_size))

  weights_for_transpose = []
  for j in range(out_channels):
    weights_in = []
    for i in range(in_channels):
      weights_in.append(np.flip(np.array(weights[j][i])))
    weights_for_transpose.append(weights_in)

  weights_for_transpose = torch.tensor(weights_for_transpose)
  weights_for_transpose = weights_for_transpose.reshape(in_channels, out_channels, kernel_size, kernel_size)

  res_tensor = []
  for l in range(out_channels):
    feature_map = np.array([])
    for i in range(0, input.shape[1] - ((weights.shape[2] - 1) * dilation + 1) + 1, stride):
      for j in range(0, input.shape[2] - ((weights.shape[3] - 1) * dilation + 1) + 1, stride):
        summa = 0
        for c in range(in_channels):
          val = input[c][i:i + (weights.shape[2] - 1) * dilation + 1:dilation,
                          j:j + (weights.shape[3] - 1) * dilation + 1:dilation]
          mini_sum = (val * weights[l][c]).sum()
          summa = summa + mini_sum
        feature_map = np.append(feature_map, float(summa + bias_val[l]))
    res_tensor.append(feature_map.reshape((input.shape[1] - ((weights.shape[2] - 1) * dilation + 1)) // stride + 1,
                                          (input.shape[2] - ((weights.shape[3] - 1) * dilation + 1)) // stride + 1))

  return np.array(res_tensor), torch.tensor(np.array(weights_for_transpose)), torch.tensor(np.array(bias_val))

In [16]:
tensor1 = torch.rand(3, 5, 6)
tensor2 = torch.rand(1, 28, 28)
tensor3 = torch.rand(7, 10, 10)

In [18]:
result, kernel, bias_val = myfunc_Conv2D(tensor1,in_channels=3, out_channels=1, kernel_size=3, transp_stride=2, bias=True,)
torchFunction = torch.nn.ConvTranspose2d(in_channels=3, out_channels=1, kernel_size=3, stride=2, bias=True,)
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val
myResult = str(np.round(result, 2))
torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))
torchResult == myResult

True

In [19]:
result, kernel, bias_val = myfunc_Conv2D(tensor2,in_channels=1, out_channels=2, kernel_size=4, transp_stride=3, bias=True)
torchFunction = torch.nn.ConvTranspose2d(in_channels=1, out_channels=2, kernel_size=4, stride=3, bias=True)
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val
myResult = str(np.round(result, 2))
torchResult = str(np.round(np.array(torchFunction(tensor2).data), 2))
torchResult == myResult

True

In [20]:
result, kernel, bias_val = myfunc_Conv2D(tensor3, in_channels=7, out_channels=1, kernel_size=3, transp_stride=5, bias=True)
torchFunction = torch.nn.ConvTranspose2d(in_channels=7, out_channels=1, kernel_size=3, stride=5, bias=True)
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val
myResult = str(np.round(result, 2))
torchResult = str(np.round(np.array(torchFunction(tensor3).data), 2))
torchResult == myResult

True